I had a run in with a sneaky feature of the Linux kernel recently.

I've recently learned about using file backed shared memory with mmap to do efficient interprocess communication. At the most basic level, you share data between two processes by writing that data to a file. The file can be made to look a lot like a normal region of memory using `mmap`, a system call, and can be shared between multiple processes by giving `mmap` the `MAP_SHARED` flag. There's a downside though, in that the data has to synced to the hard drive continuously. So, instead of sharing a file that is written to disk, I discovered that you can share a file on a RAM disk -- just a filesystem that is backed by RAM. On Linux, a large `tmpfs` RAM disk filesystem is often mounted by default at `/dev/shm`. 

Here's some python that creates a NumPy array with `mmap`. First, let's get set up...

In [18]:
import os
import mmap
n = int(1e7)
nb = 8 * n
filename = '/dev/shm/abc'

Next, we need to make sure that the file is the correct size using `ftruncate`.

In [19]:
with open(filename, 'w+b') as f:
    os.ftruncate(f.fileno(), nb)

Finally, we re-open the file and memory map it into a numpy array.

In [20]:
with open(filename, 'r+b') as f:
    mem = mmap.mmap(f.fileno(), nb)

x = np.frombuffer(mem, dtype = np.float64)

And for kicks, we'll stick some random numbers in there...

In [21]:
x[:] = np.random.rand(n)

In [14]:
print(x)

[ 0.30776649  0.24729838  0.76964428 ...,  0.34941941  0.89392064  0.7517981 ]
